In [1]:
!pip install torch torchvision transformers datasets pillow matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("nguyenkhoa/celeba-spoof-for-face-antispoofing-test")

# Print available splits
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/437 [00:00<?, ?B/s]

test-00000-of-00010.parquet:   0%|          | 0.00/496M [00:00<?, ?B/s]

test-00001-of-00010.parquet:   0%|          | 0.00/493M [00:00<?, ?B/s]

test-00002-of-00010.parquet:   0%|          | 0.00/494M [00:00<?, ?B/s]

...

test-00007-of-00010.parquet:   0%|          | 0.00/498M [00:00<?, ?B/s]

test-00008-of-00010.parquet:   0%|          | 0.00/487M [00:00<?, ?B/s]

test-00009-of-00010.parquet:   0%|          | 0.00/495M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/67170 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['cropped_image', 'labels', 'labelNames'],
        num_rows: 67170
    })
})


In [3]:
from datasets import load_dataset
import random

# Load dataset (only "test" split available)
dataset = load_dataset("nguyenkhoa/celeba-spoof-for-face-antispoofing-test")["test"]

# Shuffle dataset for randomness
dataset = dataset.shuffle(seed=42)

# Function to sample dataset
def sample_dataset(dataset, fraction):
    total_samples = int(len(dataset) * fraction)
    return dataset.select(range(total_samples))

# Use 20% for training and 10% for testing
train_data = sample_dataset(dataset, fraction=0.2)
test_data = sample_dataset(dataset, fraction=0.1)

print(f"Train Set Size: {len(train_data)}")
print(f"Test Set Size: {len(test_data)}")
print(train_data.features)
# Check if any image is missing
missing_images = [i for i in range(len(train_data)) if train_data[i]['cropped_image'] is None]
print(f"Number of missing images: {len(missing_images)}")

print(type(train_data[0]["cropped_image"]))  # Check the data type


Train Set Size: 13434
Test Set Size: 6717
{'cropped_image': Image(mode=None, decode=True, id=None), 'labels': ClassLabel(names=['live', 'spoof'], id=None), 'labelNames': Value(dtype='string', id=None)}
Number of missing images: 86
<class 'PIL.PngImagePlugin.PngImageFile'>


In [4]:
# Remove missing images from both train and test datasets
train_data = train_data.filter(lambda x: x['cropped_image'] is not None)
test_data = test_data.filter(lambda x: x['cropped_image'] is not None)

# Print new dataset size
print(f"New Train Set Size: {len(train_data)}")
print(f"New Test Set Size: {len(test_data)}")


Filter:   0%|          | 0/13434 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6717 [00:00<?, ? examples/s]

New Train Set Size: 13348
New Test Set Size: 6672


In [5]:
# Print the type and shape of a sample image
sample_image = train_data[0]["cropped_image"]

print("Type of image:", type(sample_image))

# If it's a NumPy array or list, print its shape
if isinstance(sample_image, np.ndarray) or isinstance(sample_image, list):
    print("Image shape:", np.array(sample_image).shape)


Type of image: <class 'PIL.PngImagePlugin.PngImageFile'>


NameError: name 'np' is not defined

In [ ]:
from torchvision import transforms
from PIL import Image
import torch
import numpy as np
import matplotlib.pyplot as plt

# Define image preprocessing function
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for ViT input
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Fix: Convert (C, H, W) → (H, W, C) before converting to PIL
def preprocess_data(example):
    img = example["cropped_image"]

    # If the image is a list, convert it to a NumPy array
    if isinstance(img, list):
        img = np.array(img, dtype=np.uint8)

    # Ensure it's in the correct format before converting to PIL
    if isinstance(img, np.ndarray):
        if img.shape[0] in [1, 3]:  # If shape is (C, H, W), transpose to (H, W, C)
            img = np.transpose(img, (1, 2, 0))
        img = Image.fromarray(img)  # Convert NumPy array to PIL image

    # Apply transformation
    example["cropped_image"] = transform(img)
    return example

# Apply transformation using `map()`
train_data = train_data.map(preprocess_data)
test_data = test_data.map(preprocess_data)

# Verify an image
image_tensor = train_data[0]["cropped_image"]

# Ensure the output is a tensor before displaying
if isinstance(image_tensor, torch.Tensor):
    plt.imshow(image_tensor.permute(1, 2, 0))  # Convert (C, H, W) to (H, W, C)
    plt.show()
else:
    print("Error: Image is still in the wrong format")


In [ ]:
import torch
from transformers import ViTForImageClassification

# Load pre-trained ViT model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=2  # Real (0) or Spoof (1)
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
from torch.utils.data import DataLoader, Dataset

class SpoofDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]["image"]
        label = self.dataset[idx]["label"]  # 0 = Real, 1 = Spoof
        return image, torch.tensor(label)

# Create PyTorch DataLoaders
train_loader = DataLoader(SpoofDataset(train_data), batch_size=16, shuffle=True)
test_loader = DataLoader(SpoofDataset(test_data), batch_size=16, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5)


In [ ]:
epochs = 5  # Adjust if needed

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader)}")


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        predictions = torch.argmax(outputs, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

# Compute Metrics
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary")

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


In [ ]:
from PIL import Image
import torch

# Function to predict a single image
def predict_image(image_path):
    image = Image.open(image_path)
    image_tensor = transform(image).unsqueeze(0).to(device)

    model.eval()
    output = model(image_tensor).logits
    prediction = torch.argmax(output, dim=1).item()

    label = "Real" if prediction == 0 else "Spoof"
    print(f"Prediction for {image_path}: {label}")

# Test real and spoofed images
predict_image("real.jpg")
predict_image("spoof.jpg")
